# On modelling optimization problems via Julia JuMP

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Department of Computer Science**  
**Lavras, Minas Gerais, Brazil**  

*Università degli Studi di Modena e Reggio Emilia (UNIMORE)*  
*Reggio Emilia, Italy*

## JuMP: Julia for Mathematical Programming

### Problem 3: Lot-sizing mono-stage

* This example was adapted from ["Arenales, M., Morabito, R., Armentano, V., & Yanasse, H. (2015). Pesquisa operacional: para cursos de engenharia. Elsevier Brasil."](https://www.amazon.com.br/Pesquisa-Operacional-Para-cursos-engenharia-ebook/dp/B016APOY5U "Arenales et al. (2015): \"PO para Engenharia\"")

Consider a company that manufactures $n$ products and wants to schedule its production over the next $T$ periods. Each product has a specific demand in each period, which must be exactly met. The costs of production (stock) depend on the quantity produced (stored) and vary according to the item and period. The production of a particular item consumes a certain amount of resource (e.g., water, energy, gas) pre-determined for the period in question. Given this scenario, this enterprise wants to plan the production control of the products in each period, aiming at minimizing production costs and inventory costs.

#### Mathematical modeling

* **Parameters**
    * $n$: number of products;
    * $T$: number of periods;
    * $d_{it}$: demand of item $i$ in the period $t$, $i=1,...,n$ and $t=1,...,T$;
    * $R_t$: availability of resources (renewable) in the period $t$;
    * $r_i$: amount of resources needed for the production of one unit of item $i$;
    * $c_{it}$: production cost for one unit of item $i$ in the period $t$;
    * $h_{it}$: inventory costs of item $i$ in the period $t$.
    
* **Variables**
    * $x_{it} \in \mathbb{Z}_+$: quantity of item $i$ produced at period $t$;
    * $I_{it} \in \mathbb{Z}_+$: quantity of item $i$ stored at period $t$.
    
* **Formulation**

\begin{equation}
\min f(x,I) = \sum_{i=1}^{n}\sum_{t=1}^T c_{it}x_{it} + \sum_{i=1}^{n}\sum_{t=1}^T h_{it}I_{it}
\end{equation}

suject to:

\begin{equation}
I_{i,t-1} + x_{it} - I_{it} = d_{it} \qquad i=1,...,n,t=1,...,T
\end{equation}

\begin{equation}
\sum_{i=1}^{n}r_{i}x_{it} \le R_t \qquad t=1,...,T
\end{equation}

\begin{equation}
x_{it} \in \mathbb{Z}_+ \qquad i=1,...,n,t=1,...,T
\end{equation}

\begin{equation}
I_{it} \in \mathbb{Z}_+ \qquad i=1,...,n,t=1,...,T
\end{equation}

* **Parameters in Julia**

In [1]:
# Number of items
n = 3

# Number of periods
T = 2

# Production costs (another way to declare a matrix)
c = [[100,80],
     [60,80],
     [30,200]]

# Inventory costs
h = [[2,2.5],
     [3,3.5],
     [3.5,3.5]]

# Resource vector
r = [0.25,0.3,0.3]

# Capacity of each period
R = [200,250]

# Demands for each item, in each period
d = [[50,20],
     [40,60],
     [100,80]]

3-element Array{Array{Int64,1},1}:
 [50, 20] 
 [40, 60] 
 [100, 80]

Sometimes we will need to read input files by a spreadsheet. For that purpose, I suggest to the reader "*DataFrame*" package: https://juliadata.github.io/DataFrames.jl/stable/man/getting_started.html.

* **OR libraries**

In [2]:
using JuMP, Cbc

* **Object Model**

In [3]:
model = Model(with_optimizer(Cbc.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

* **Variables**

In [4]:
@variable(model, x[1:n, 1:T] >= 0, Int)
@variable(model, I[1:n, 1:T] >= 0, Int)

3×2 Array{VariableRef,2}:
 I[1,1]  I[1,2]
 I[2,1]  I[2,2]
 I[3,1]  I[3,2]

* **Objective function**

In [5]:
@objective(model, Min, sum(c[i][t]*x[i,t] + h[i][t]*I[i,t] for i=1:n for t=1:T))

100 x[1,1] + 2 I[1,1] + 80 x[1,2] + 2.5 I[1,2] + 60 x[2,1] + 3 I[2,1] + 80 x[2,2] + 3.5 I[2,2] + 30 x[3,1] + 3.5 I[3,1] + 200 x[3,2] + 3.5 I[3,2]

* **Constraints**

In [6]:
@constraints(model, begin
        # Inventory control; Demands must be attended
        [i=1:n,t=2:T], I[i,t-1] + x[i,t] - I[i,t] == d[i][t]  
        # Inventory control; Demands must be attended (For the first period)
        [i=1:n], x[i,1] - I[i,1] == d[i][1]
        # Capacity (knapsack) constraints
        [t=1:T], sum(r[i]*x[i,t] for i=1:n) <= R[t]
        end)

* **Optimize it! Then we print the solution**

In [8]:
start = time()
optimize!(model)

println("Time: $(time()-start)s")
println("Objective function = ", objective_value(model))

println("x")
for i=1:n
    for t=1:T
        if(value(x[i,t]) > 0)
            println("x[", i, ",", t, "]: ", value(x[i,t]))
        end
    end
end

println("I")
for i=1:n
    for t=1:T
        if(value(I[i,t]) > 0)
            println("I[", i, ",", t, "]: ", value(I[i,t]))
        end
    end
end

Time: 0.03400015830993652s
Objective function = 18460.0
x
x[1,1]: 50.0
x[1,2]: 20.0
x[2,1]: 100.0
x[3,1]: 180.0
I
I[2,1]: 60.0
I[3,1]: 80.0
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 18460 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cbc3007W No integer variables - nothing to do
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0045I Solution of 18460 already found by heuristic
Cuts at root node changed objective from 1.79769e+308 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after add